<a href="https://colab.research.google.com/github/terry-ab/NER-BILSTM/blob/master/NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import optimizers
from keras.layers import LSTM,SimpleRNN,Bidirectional,TimeDistributed
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.preprocessing.sequence import pad_sequences
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support
from keras import backend as K
import numpy as np
import re
from sklearn import preprocessing
from gensim.models import Word2Vec
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import keras
from keras.utils import to_categorical

# New Section

In [0]:
df=pd.read_csv('/content/drive/My Drive/ner.csv',encoding = "ISO-8859-1",error_bad_lines=False)

In [0]:
(df.word[:50000],df.tag[:50000])

(0            Thousands
 1                   of
 2        demonstrators
 3                 have
 4              marched
              ...      
 49995           pushed
 49996               by
 49997            South
 49998           Africa
 49999                ,
 Name: word, Length: 50000, dtype: object, 0            O
 1            O
 2            O
 3            O
 4            O
          ...  
 49995        O
 49996        O
 49997    B-geo
 49998    I-geo
 49999        O
 Name: tag, Length: 50000, dtype: object)

In [0]:
df=df.drop(columns=['Unnamed: 3','Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10',
       'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14',
       'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18',
       'Unnamed: 19'])
df

,Unnamed: 0,pos,sentence_idx,word,tag
0,0,NNS,1,Thousands,O
1,1,IN,1,of,O
2,2,NNS,1,demonstrators,O
3,3,VBP,1,have,O
4,4,VBN,1,marched,O
...,...,...,...,...,...
100017,100017,NNS,4544,stones,O
100018,100018,IN,4544,at,O
100019,100019,NNP,4544,U.S.,B-geo
100020,100020,NNS,4544,troops,O


##Bi-lstm

In [0]:
 #importing pretrained word2vec model
 import gensim
 wvmodel=gensim.models.KeyedVectors.load_word2vec_format('https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz',binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
#initializing words and tags to a list
words = list(set(df["word"][:50000].values))
#adding endpad for padding purpose
words.append("ENDPAD")
n_words = len(words)
tags = list(set(df["tag"][:50000].values))
n_tags = len(tags)

In [0]:
#We use SentenceGetter function to retrive the sentence from sentence_id for padding.it will return in format of (word,pos,tag)
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["word"].values.tolist(),
                                                           s["pos"].values.tolist(),
                                                           s["tag"].values.tolist())]
        self.grouped = self.data.groupby("sentence_idx").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [0]:
getter = SentenceGetter(df[:50000])
sent = getter.get_next()
sentences = getter.sentences

In [0]:
#setting maxlen and index for words and tags
max_len = 50
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}

In [0]:
word2idx

{'roadside': 0,
 'remove': 1,
 'bulk': 2,
 'raw': 3,
 'Tony': 4,
 'boosted': 5,
 'explicitly': 6,
 'Arabic': 7,
 'territorial': 8,
 'helicopter': 9,
 'al-Zawahri': 10,
 '\x85': 11,
 'hazelnuts': 12,
 'designed': 13,
 'potential': 14,
 'risen': 15,
 'downplaying': 16,
 'Quami': 17,
 "Shi'ites": 18,
 'individuals': 19,
 'fifth': 20,
 'Families': 21,
 'apartment': 22,
 'bordered': 23,
 'Jack': 24,
 'Army': 25,
 'invitation': 26,
 'renew': 27,
 'Oklahoma': 28,
 'panic': 29,
 'Indonesia': 30,
 '26': 31,
 'kidnappers': 32,
 'Stanizai': 33,
 'empty': 34,
 'screen': 35,
 'resolve': 36,
 'safely': 37,
 'homeless': 38,
 'ceremony': 39,
 'MacArthur': 40,
 'vaccinations': 41,
 'war': 42,
 'deposed': 43,
 'He': 44,
 'Thomas': 45,
 'collect': 46,
 'evil': 47,
 'Station': 48,
 'advance': 49,
 'disputes': 50,
 'buildings': 51,
 'Nordic': 52,
 'Seven': 53,
 'Helmut': 54,
 'Valencia': 55,
 'Viktor': 56,
 'sanctions': 57,
 'Cracks': 58,
 'electronically': 59,
 'sale': 60,
 'sectors': 61,
 'website': 62,


In [0]:
#padding
from keras.preprocessing.sequence import pad_sequences
X = [[word2idx[w[0]] for w in s] for s in sentences]
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words - 1)

In [0]:
y = [[tag2idx[w[2]] for w in s] for s in sentences]
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])
y[0]

array([15, 15, 15, 15, 15, 15,  3, 15, 15, 15, 15, 15,  3, 15, 15, 15, 15,
       15, 14, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15,
       15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15],
      dtype=int32)

In [0]:
X[0]

array([3873, 1756, 2046, 4793, 4223, 3850, 1532, 6043, 6447, 2005,   42,
       4778, 2715, 5770, 3760, 2005,  647, 1756, 6890, 6517, 2711,  911,
       5706, 4075, 7464, 7464, 7464, 7464, 7464, 7464, 7464, 7464, 7464,
       7464, 7464, 7464, 7464, 7464, 7464, 7464, 7464, 7464, 7464, 7464,
       7464, 7464, 7464, 7464, 7464, 7464], dtype=int32)

In [0]:
from keras.utils import to_categorical
y = [to_categorical(i, num_classes=n_tags) for i in y]

In [0]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1)

In [0]:
#setting embedding matrix for weight in keras Embedding layer
embedding_matrix=np.zeros((len(word2idx)+1,300))

for word,i in word2idx.items():
    try:
        embedding_vector=wvmodel[word]
        embedding_matrix[i]=embedding_vector
        
    except KeyError:
        pass

In [0]:
#bi-lstm
model=Sequential()
model.add(Embedding(input_dim=len(word2idx) + 1, output_dim=300,  weights=[embedding_matrix],input_length=max_len,))
model.add(Dropout(0.1))
model.add(Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1)))
model.add(Dense(n_tags, activation="softmax"))  # softmax output layer











Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 300)           2239800   
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 300)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 50, 200)           320800    
_________________________________________________________________
dense_1 (Dense)              (None, 50, 17)            3417      
Total params: 2,564,017
Trainable params: 2,564,017
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [0]:
model.fit(X_tr, np.array(y_tr), batch_size=32, epochs=5, validation_split=0.1, verbose=1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 1838 samples, validate on 205 samples
Epoch 1/5
1838/1838 [==============================] - 18s 10ms/step - loss: 0.7551 - acc: 0.9174 - val_loss: 0.3037 - val_acc: 0.9335
Epoch 2/5
1838/1838 [==============================] - 15s 8ms/step - loss: 0.2277 - acc: 0.9446 - val_loss: 0.1649 - val_acc: 0.9579
Epoch 3/5
1838/1838 [==============================] - 15s 8ms/step - loss: 0.1294 - acc: 0.9679 - val_loss: 0.1099 - val_acc: 0.9706
Epoch 4/5
1838/1838 [==============================] - 15s 8ms/step - loss: 0.0880 - acc: 0.9765 - val_loss: 0.0891 - val_acc: 0.9757
Epoch 5/5
1838/1838 [==============================] - 15s 8ms/step - loss: 0.0665 - acc: 0.9810 - val_loss: 0.0815 - val_acc: 0.9769


In [0]:
score = model.evaluate(X_te,np.array(y_te))
print("Accuracy = ",score[1]*100)

227/227 [==============================] - 0s 1ms/step
Accuracy =  97.45374398084465


In [0]:
tr_score=model.evaluate(X_tr,np.array(y_tr))
print("Accuracy = ",tr_score[1]*100)

2043/2043 [==============================] - 3s 1ms/step
Accuracy =  98.3690649373124


In [0]:
i = 10
p = model.predict_classes(np.array([X_te[i]]))
for w, pred in zip(X_te[i], p[0]):
    print("{:5}: {}".format(words[w], tags[pred]))

Simonyi: B-per
brought: O
with : O
him  : O
a    : O
gourmet: O
dinner: O
to   : O
be   : O
eaten: O
Thursday: B-tim
,    : O
which: O
Russia: B-geo
marks: O
as   : O
Cosmonauts: O
Day  : B-tim
--   : O
the  : O
anniversary: O
of   : O
Yuri : B-per
Gagarin: I-per
's   : O
first: O
manned: O
space: O
flight: O
in   : O
1961 : B-tim
.    : O
ENDPAD: O
ENDPAD: O
ENDPAD: O
ENDPAD: O
ENDPAD: O
ENDPAD: O
ENDPAD: O
ENDPAD: O
ENDPAD: O
ENDPAD: O
ENDPAD: O
ENDPAD: O
ENDPAD: O
ENDPAD: O
ENDPAD: O
ENDPAD: O
ENDPAD: O
ENDPAD: O
